###***Accessing Authenticity and Providence of Online Reviews in Travel and Leisure Industry***

In [ ]:
# Importing the required libraries
import string
import warnings
import re
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import seaborn as sns
import nltk
from textblob import TextBlob
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from collections import Counter
from nltk.stem import PorterStemmer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB

In [ ]:
# downloading the required pakages
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('vader_lexicon')
nltk.download('omw-1.4')

In [ ]:
# specifying the path
Real_file = '/content/drive/MyDrive/Colab Notebooks/Real_RedDragonReviews.xlsx'
fake_file= '/content/drive/MyDrive/Colab Notebooks/Fake_RedDragonReviews.xlsx'

In [ ]:
# loading the files
df1 = pd.read_excel(Real_file, sheet_name='Sheet1')
df2 = pd.read_excel(fake_file, sheet_name='Sheet1')

In [ ]:
df1

In [ ]:
df2

In [ ]:
# Merging the two dataframes
df_merged= pd.concat([df1, df2], ignore_index=True)

In [ ]:
# Save the merged dataframe to a new CSV file
df_merged.to_csv('/content/drive/MyDrive/Colab Notebooks/RedDragon_Reviews.csv', index=False)

In [ ]:
# loading the saved file
df=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/RedDragon_Reviews.csv')

In [ ]:
# first five records
df.head()

In [ ]:
# last five records
df.tail()

#***Data Pre-Processing***

In [ ]:
# no.of rows and columns
df.shape

In [ ]:
# displaying the information about the dataset
print("Basic information on the dataset:")
print(df.info())

In [ ]:
# Remove duplicates
df = df.drop_duplicates()

In [ ]:
# checking for the missing values
df.isnull().sum()

In [ ]:
# filling the missing values for values
mode_value = df['value'].mode()[0]
rounded_mode_value = round(mode_value)
df['value'] = df['value'].fillna(rounded_mode_value)

In [ ]:
# filling the missing values for service
mode_value = df['service'].mode()[0]
rounded_mode_value = round(mode_value)
df['service'] = df['service'].fillna(rounded_mode_value)

In [ ]:
# filling the missing values for location
mode_value = df['location'].mode()[0]
rounded_mode_value = round(mode_value)
df['location'] = df['location'].fillna(rounded_mode_value)

In [ ]:
# filling the missing values for rooms
mode_value = df['rooms'].mode()[0]
rounded_mode_value = round(mode_value)
df['rooms'] = df['rooms'].fillna(rounded_mode_value)

In [ ]:
# filling the missing values for cleanliness
mode_value = df['cleanliness'].mode()[0]
rounded_mode_value = round(mode_value)
df['cleanliness'] = df['cleanliness'].fillna(rounded_mode_value)

In [ ]:
# filling the missing values for sleep_quality
mode_value = df['sleep_quality'].mode()[0]
rounded_mode_value = round(mode_value)
df['sleep_quality'] = df['sleep_quality'].fillna(rounded_mode_value)

In [ ]:
# creating a new coloumn label_encoded with values for 'OR'=0 and 'AI'=1
df['label_encoded'] = df['label'].apply(lambda x: 1 if x == 'AI' else 0)
df['label_encoded']

In [ ]:
# Initialize stop words, stemmer, and lemmatizer for text preprocessing
stop_words = set(stopwords.words('english'))
stop_words.update([
    'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your',
    'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it',
    "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this',
    'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had',
    'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until',
    'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before',
    'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',
    'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most',
    'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can',
    'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren',
    "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven',
    "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't",
    'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"
])

# Initialize stemmer and lemmatizer
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

In [ ]:
# # function to preprocess the text
# def preprocess_text(text):
#     text = text.lower()  #converting the text to lower case
#     text = text.translate(str.maketrans('', '', string.punctuation)) #Removing the punctuation
#     text = re.sub(r'\d+', '', text)  # Removing numbers
#     text = re.sub(r"\W", " ", text)         # Replace non-word characters with a space
#     text = re.sub(r"\d", " ", text)         # Replace digits with a space
#     text = re.sub(r"\s+[a-z]\s+", " ", text, flags=re.I)  # Replace single letters with a space
#     text = re.sub(r"\s+", " ", text)        # Replace multiple spaces with a single space
#     text = re.sub(r"^\s", "", text)         # Remove leading whitespace
#     text = re.sub(r"\s$", "", text)         # Remove trailing whitespace
#     words = word_tokenize(text) #Tokenizing the text
#     words = [word for word in words if word not in stop_words] # removeing the stop words
#     return ' '.join(words)

In [ ]:
 # Apply the preprocessing to the review column
# df['processed_review'] = df['review'].apply(preprocess_text)

In [ ]:
# df['processed_review']

#***Stemming***

In [ ]:
def preprocess_text(text):
    # Remove numbers from the text
    text = re.sub(r'[^a-zA-Z]', ' ', text)
    # Lowercase the text
    text = text.lower()
    # Remove punctuation from text
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Tokenize the text
    words = word_tokenize(text)
    # Remove stopwords and apply stemming to the text
    words = [stemmer.stem(word) for word in words if word not in stop_words]
    return ' '.join(words)

In [ ]:
# Apply the preprocessing to the review column
df['processed_review'] = df['review'].apply(preprocess_text)

In [ ]:
df[['review','processed_review']]

#**Lemmatization**

In [ ]:
def preprocess_text(text):
    # remove numbers from the text
    text = re.sub(r'[^a-zA-Z]', ' ', text)
    # Lowercase the text
    text = text.lower()
    # remove punctuation from the text
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Tokenize the text
    words = word_tokenize(text)
    # Remove stopwords and lemmatize the words
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    return ' '.join(words)

In [ ]:
# Apply the preprocessing to the review column
df['processed_review'] = df['review'].apply(preprocess_text)

In [ ]:
df[['review','processed_review']]

In [ ]:
# def lemmatize_text(text):
#     words = word_tokenize(text)
#     lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
#     return ' '.join(lemmatized_words)

In [ ]:
# # Apply the preprocessing to the review column
# df['processed_review'] = df['processed_review'].apply(lemmatize_text)

In [ ]:
# df['processed_review'].head()

In [ ]:
# df['processed_review'].tail()

In [ ]:
#  Function to perform POS tagging
# def pos_tagging(text):
#     words = word_tokenize(text)
#     return pos_tag(words)

In [ ]:
# Apply POS tagging to the processed reviews
#df['pos_tags'] = df['processed_review'].apply(pos_tagging)

In [ ]:
#df['pos_tags']

In [ ]:
# number of words in sentence
sent_len = [] #creating an empty list
for sent in df['processed_review']:
  sent_len.append(len(word_tokenize(sent)))
df['sent_len'] = sent_len
df['num_chars'] = df['processed_review'].apply(len)# Number of characters
df['avg_word_len'] = df['processed_review'].apply(lambda x: sum(len(word) for word in x.split()) / len(x.split()))# Average word length
df['num_unique_words'] = df['processed_review'].apply(lambda x: len(set(x.split())))# Number of unique words

In [ ]:
# Number of nouns, verbs, and adjectives
def count_pos(text):
    words = word_tokenize(text)
    pos_tags = nltk.pos_tag(words)
    pos_counts = Counter(tag for word, tag in pos_tags)
    return pos_counts['NN'], pos_counts['VB'], pos_counts['JJ']  # Nouns, Verbs, Adjectives

In [ ]:
# Apply the function and get a list of tuples
pos_counts = df['processed_review'].apply(count_pos)

# Convert the list of tuples into a DataFrame
pos_df = pd.DataFrame(pos_counts.tolist(), columns=['num_nouns', 'num_verbs', 'num_adjs'])

# Concatenate the original DataFrame with the new POS DataFrame
df = pd.concat([df, pos_df], axis=1)

In [ ]:
df[['sent_len', 'num_chars','avg_word_len','num_unique_words','num_nouns','num_nouns','num_verbs','num_adjs']]

#***Exploratory Data Analysis***

In [ ]:
# Count plot of Rating by OR v/s AI
plt.figure(figsize=(10, 6))
sns.countplot(data=df, x='rating', hue='label', palette='flare')
plt.title('Count Plot of Overall Ratings Provided by OR v/s AI',fontweight='bold',fontstyle='italic')
plt.xlabel('Rating')
plt.ylabel('Count')
plt.legend(title='Label', loc='upper left')
plt.show()

In [ ]:
# Filter the data based on label, for 'OR'
df_filtered_or = df[df['label'] == 'OR']

# Conversion of the DataFrame to long format for 'OR'
df_or = df_filtered_or.melt(id_vars=['label'], value_vars=['value', 'service', 'location', 'rooms', 'cleanliness', 'sleep_quality'],
                                   var_name='category', value_name='rating_value')

# Plot the data for 'OR'
plt.figure(figsize=(10, 6))
sns.countplot(data=df_or, x='category', hue='rating_value', palette='colorblind')
plt.title('Count Plot of Ratings for Different Sectors Provided by OR',fontweight='bold',fontstyle='italic')
plt.xlabel('Category')
plt.ylabel('Count')
plt.legend(title='Rating', loc='upper right')
plt.show()

In [ ]:
# Filter the data based on label, for 'AI'
df_filtered_ai = df[df['label'] == 'AI']

# conversion of the DataFrame to long format for 'AI'
df_ai = df_filtered_ai.melt(id_vars=['label'], value_vars=['value', 'service', 'location', 'rooms', 'cleanliness', 'sleep_quality'],
                                   var_name='category', value_name='rating_value')

# Plot the data for 'AI'
plt.figure(figsize=(10, 6))
sns.countplot(data=df_ai, x='category', hue='rating_value', palette='viridis')
plt.title('Count Plot of Ratings for Different Categories Provided by AI',fontweight='bold',fontstyle='italic')
plt.xlabel('Category')
plt.ylabel('Count')
plt.legend(title='Rating', loc='upper right')
plt.show()

In [ ]:
# Plot to check if the dataset is balanced based on the tareget varaibale'label' column
plt.figure(figsize=(8, 6))
sns.countplot(data=df, x='label', palette='viridis')
plt.title('Count Plot to Check distribution of the target variable (label)',fontweight='bold',fontstyle='italic')
plt.xlabel('Label')
plt.ylabel('Count')
plt.show()

In [ ]:
# overall distribution of rating percentage for both "OR" & "AI"
plt.pie(df['rating'].value_counts(), labels=df['rating'].unique().tolist(), autopct='%1.1f%%')
plt.title('Overall Rating Distribution in percentage',fontweight='bold',fontstyle='italic')
plt.show()

#***Sentiment Analysis***

#***TextBlob***

In [ ]:
#installing the TextBlob library
!pip install textblob

In [ ]:
#importing the library
from textblob import TextBlob

In [ ]:
# Function to get  polarity
def get_polarity(text):
    polarity = TextBlob(text).sentiment.polarity
    return int(round(polarity))

# Function to get subjectivity
def get_subjectivity(text):
    subjectivity = TextBlob(text).sentiment.subjectivity
    return int(round(subjectivity))

In [ ]:
df['textblob_polarity'] = df['processed_review'].apply(get_polarity)
df['textblob_subjectivity'] = df['processed_review'].apply(get_subjectivity)

In [ ]:
df[['label','textblob_polarity','textblob_subjectivity']]

**Visualization of Polarity and subjectivity**

In [ ]:
# Filter the DataFrame for 'AI' and 'OR' reviews
df_ai = df[df['label'] == 'AI']
df_or = df[df['label'] == 'OR']

# Create a figure with 2 rows and 2 columns of subplots
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Plotting Polarity for OR
sns.countplot(x='textblob_polarity', data=df_or, palette='magma', ax=axes[0, 0])
axes[0, 0].set_title('Polarity Distribution for OR Reviews',fontweight='bold',fontstyle='italic')
axes[0, 0].set_xlabel('Polarity')
axes[0, 0].set_ylabel('Count')

# Plotting Polarity for AI
sns.countplot(x='textblob_polarity', data=df_ai, palette='viridis', ax=axes[0, 1])
axes[0, 1].set_title('Polarity Distribution for AI Reviews',fontweight='bold',fontstyle='italic')
axes[0, 1].set_xlabel('Polarity')
axes[0, 1].set_ylabel('Count')

# Plotting polarity for OR
sns.countplot(x='textblob_subjectivity', data=df_or, palette='bright', ax=axes[1, 0])
axes[1, 0].set_title('Subjectivity Distribution for OR Reviews',fontweight='bold',fontstyle='italic')
axes[1, 0].set_xlabel('Subjectivity')
axes[1, 0].set_ylabel('Count')

# Plotting polarity for AI
sns.countplot(x='textblob_subjectivity', data=df_ai, palette='Greens', ax=axes[1, 1])
axes[1, 1].set_title(' Subjectivity Distribution for AI Reviews',fontweight='bold',fontstyle='italic')
axes[1, 1].set_xlabel('Subjectivity')
axes[1, 1].set_ylabel('Count')

# Adjust layout to prevent overlap
plt.tight_layout()
plt.show()

#***VADER (Valence Aware Dictionary and sEntiment Reasoner)***

In [ ]:
# Installation of VADER Sentiment library
!pip install vaderSentiment

In [ ]:
# Initialize the VADER sentiment analyzer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

In [ ]:
# Function to get sentiment scores and labels
def get_vader_analysis(text):
    sentiment_dict = analyzer.polarity_scores(text)
    sentiment_label = 'positive' if sentiment_dict['compound'] >= 0.05 else ('negative' if sentiment_dict['compound'] <= -0.05 else 'neutral')
    return pd.Series([sentiment_dict['neg'], sentiment_dict['neu'], sentiment_dict['pos'], sentiment_dict['compound'], sentiment_label])

In [ ]:
# Apply VADER sentiment analysis to the processed reviews
df[['vader_neg', 'vader_neu', 'vader_pos', 'compound', 'vader_sentiment_label']] = df['processed_review'].apply(get_vader_analysis)

In [ ]:
# Print the VADER sentiment scores and labels
df[['processed_review', 'vader_neg', 'vader_neu', 'vader_pos', 'compound', 'vader_sentiment_label']]

In [ ]:
# Count plot of VADER sentiment labels by OR vs. AI
plt.figure(figsize=(10, 6))
sns.countplot(data=df, x='vader_sentiment_label', hue='label', palette='viridis')
plt.title('VADER Sentiment Distribution for OR vs. AI Reviews', fontweight='bold', fontstyle='italic')
plt.xlabel('Sentiment')
plt.ylabel('Count')
plt.legend(title='Label', loc='upper right')
plt.show()

#***RQ2: “How does the sentiments of reviews varies between genuine and fake reviews?***

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
#scipy used to perform independent t-test
from scipy.stats import ttest_ind

In [ ]:
# Filter the reviews based on 'OR' (genuine) and 'AI' (fake) labels
Original_reviews = df[df['label'] == 'OR']['compound']
AI_reviews = df[df['label'] == 'AI']['compound']

In [ ]:
# Descriptive Statistics
print(f"Genuine Reviews - Mean: {Original_reviews.mean()}, Std Dev: {Original_reviews.std()}")
print(f"AI Reviews - Mean: {AI_reviews.mean()}, Std Dev: {AI_reviews.std()}")

In [ ]:
# Perform t-test
t_statistic, p_value = ttest_ind(Original_reviews, AI_reviews)
print(f"T-statistic: {t_statistic}, P-value: {p_value}")

In [ ]:
# Visualization: Boxplot
plt.figure(figsize=(10, 6))
plt.boxplot([Original_reviews, AI_reviews], labels=['Original Reviews', 'AI Reviews'])
plt.title('Sentiment Score Distribution',fontweight='bold',fontstyle='italic')
plt.ylabel('Compound Sentiment Score')
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
sns.kdeplot(Original_reviews, shade=True, label='Original Reviews', color='blue')
sns.kdeplot(AI_reviews, shade=True, label='AI Reviews', color='red')
plt.title('Density Plot of Sentiment Scores', fontweight='bold', fontstyle='italic')
plt.xlabel('Compound Sentiment Score')
plt.ylabel('Density')
plt.legend()
plt.show()

#***RQ3: What do the word patterns shown in the WordClouds reveals the opinions between fake and genuine reviews?***

#***WordCloud Visualization***

#***VADER***

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from wordcloud import WordCloud

In [ ]:
# Function to generate word cloud for postive review
def generate_wordcloud(text, ax, title, background_color):
    wordcloud = WordCloud(width=800, height=800, background_color=background_color, colormap='viridis', min_font_size=10).generate(text)
    ax.imshow(wordcloud, interpolation="bilinear")
    ax.axis("off")
    ax.set_title(title, fontsize=16)  # Reduced font size

# Create figure and axes for positive reviews
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=[10, 5], facecolor='white')  # Adjusted figure size and facecolor

# Positive Reviews
text_pos_or = ' '.join(df[(df['label'] == 'OR') & (df['vader_sentiment_label'] == 'positive')]['processed_review'])
generate_wordcloud(text_pos_or, ax1, 'OR - Positive Reviews', background_color='white')

text_pos_ai = ' '.join(df[(df['label'] == 'AI') & (df['vader_sentiment_label'] == 'positive')]['processed_review'])
generate_wordcloud(text_pos_ai, ax2, 'AI - Positive Reviews', background_color='black')

# Adjust layout
plt.tight_layout()
plt.show()

In [ ]:
# Function to generate word cloud for negative reviews
def generate_wordcloud(text, ax, title, background_color):
    wordcloud = WordCloud(width=800, height=800, background_color=background_color, colormap='cool', min_font_size=10).generate(text)
    ax.imshow(wordcloud, interpolation="bilinear")
    ax.axis("off")
    ax.set_title(title, fontsize=16)  # Reduced font size

# Create figure and axes for negative reviews
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=[10, 5], facecolor='white')  # Adjusted figure size and facecolor

# Negative Reviews
text_neg_or = ' '.join(df[(df['label'] == 'OR') & (df['vader_sentiment_label'] == 'negative')]['processed_review'])
generate_wordcloud(text_neg_or, ax1, 'OR - Negative Reviews', background_color='lightblue')

text_neg_ai = ' '.join(df[(df['label'] == 'AI') & (df['vader_sentiment_label'] == 'negative')]['processed_review'])
generate_wordcloud(text_neg_ai, ax2, 'AI - Negative Reviews', background_color='lightpink')

# Adjust layout
plt.tight_layout()
plt.show()

In [ ]:
# Function to generate word cloud for neutral reviews
def generate_wordcloud(text, ax, title, background_color):
    wordcloud = WordCloud(width=800, height=800, background_color=background_color, colormap='plasma', min_font_size=10).generate(text)
    ax.imshow(wordcloud, interpolation="bilinear")
    ax.axis("off")
    ax.set_title(title, fontsize=16)  # Reduced font size

# Create figure and axes for neutral reviews
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=[10, 5], facecolor='white')  # Adjusted figure size and facecolor

# Neutral Reviews
text_neu_or = ' '.join(df[(df['label'] == 'OR') & (df['vader_sentiment_label'] == 'neutral')]['processed_review'])
generate_wordcloud(text_neu_or, ax1, 'OR - Neutral Reviews', background_color='green')

text_neu_ai = ' '.join(df[(df['label'] == 'AI') & (df['vader_sentiment_label'] == 'neutral')]['processed_review'])
generate_wordcloud(text_neu_ai, ax2, 'AI - Neutral Reviews', background_color='gray')

# Adjust layout
plt.tight_layout()
plt.show()

#***Feature Engineering***

#***Unigram***

In [ ]:
# Vectorize the processed reviews using unigrams
vectorizer = CountVectorizer(ngram_range=(1, 1))
X = vectorizer.fit_transform(df['processed_review'])

In [ ]:
# Convert the unigrams into a DataFrame
unigram_df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())

In [ ]:
# Merge the unigram features with the original DataFrame
df_unigrams = pd.concat([df, unigram_df], axis=1)

In [ ]:
# Ensure that we only have numeric columns in unigram_features by filtering explicitly
unigram_features = unigram_df.columns.tolist()

In [ ]:
# Filter out only numeric columns for processing
numeric_unigram_features = [col for col in unigram_features if pd.api.types.is_numeric_dtype(df_unigrams[col])]

In [ ]:
# Calculate the sum of each unigram for AI reviews
unigram_ai_sums = df_unigrams[df_unigrams['label_encoded'] == 1][numeric_unigram_features].sum().sort_values(ascending=False)

# Calculate the sum of each unigram for OR reviews
unigram_or_sums = df_unigrams[df_unigrams['label_encoded'] == 0][numeric_unigram_features].sum().sort_values(ascending=False)

# Select the top 20 unigrams for OR reviews
top_n_unigrams_or = unigram_or_sums.head(20)

# Select the top 20 unigrams for OR reviews
top_n_unigrams_ai = unigram_ai_sums.head(20)

In [ ]:
# Adjust threshold if necessary
threshold = top_n_unigrams_or.min() + 1

# Filter out unigrams with low frequencies (set threshold as needed)
top_n_unigrams_or_filtered = top_n_unigrams_or[top_n_unigrams_or > threshold]

In [ ]:
# Plot the filtered top 20 unigrams for OR reviews
plt.figure(figsize=(12, 8))
sns.barplot(x=top_n_unigrams_or_filtered.values, y=top_n_unigrams_or_filtered.index, palette="viridis")
plt.title('Top 20 Most Frequent Unigrams in OR Reviews',fontweight='bold', fontstyle='italic')
plt.xlabel('Frequency')
plt.ylabel('Unigrams')
plt.show()

In [ ]:
# Adjust threshold if necessary
threshold = top_n_unigrams_ai.min() + 1

# Filter out unigrams with low frequencies (set threshold as needed)
top_n_unigrams_ai_filtered = top_n_unigrams_ai[top_n_unigrams_ai > threshold]

In [ ]:
# Plot the filtered top 20 unigrams for OR reviews
plt.figure(figsize=(12, 8))
sns.barplot(x=top_n_unigrams_ai_filtered.values, y=top_n_unigrams_ai_filtered.index, palette="cividis")
plt.title('Top 20 Most Frequent Unigrams in AI Reviews',fontweight='bold', fontstyle='italic')
plt.xlabel('Frequency')
plt.ylabel('Unigrams')
plt.show()

#***Bigram***

In [ ]:
# Vectorize the processed reviews using bigrams
vectorizer = CountVectorizer(ngram_range=(2, 2))
X = vectorizer.fit_transform(df['processed_review'])

In [ ]:
# Convert the bigrams into a DataFrame
bigram_df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())

In [ ]:
# Merge the bigram features with the original DataFrame
df_bigrams = pd.concat([df, bigram_df], axis=1)

In [ ]:
# Define the feature columns, including bigram features
bigram_features = bigram_df.columns.tolist()

In [ ]:
# Calculate the sum of each bigram for AI reviews
bigram_ai_sums = df_bigrams[df_bigrams['label_encoded'] == 1][bigram_df.columns].sum().sort_values(ascending=False)

# Calculate the sum of each bigram for OR reviews
bigram_or_sums = df_bigrams[df_bigrams['label_encoded'] == 0][bigram_df.columns].sum().sort_values(ascending=False)

# Select the top 20 bigrams for AI reviews
top_n_bigrams_ai = bigram_ai_sums.head(20)

# Select the top 20 bigrams for OR reviews
top_n_bigrams_or = bigram_or_sums.head(20)

In [ ]:
# Plot the top 20 bigrams for OR reviews
plt.figure(figsize=(12, 8))
sns.barplot(x=top_n_bigrams_or.values, y=top_n_bigrams_or.index, palette="plasma")
plt.title('Top 20 Most Frequent Bigrams in OR Reviews',fontweight='bold', fontstyle='italic')
plt.xlabel('Frequency')
plt.ylabel('Bigrams')
plt.show()

In [ ]:
# Plot the top 20 bigrams for AI reviews
plt.figure(figsize=(12, 8))
sns.barplot(x=top_n_bigrams_ai.values, y=top_n_bigrams_ai.index, palette="inferno")
plt.title('Top 20 Most Frequent Bigrams in AI Reviews',fontweight='bold', fontstyle='italic')
plt.xlabel('Frequency')
plt.ylabel('Bigrams')
plt.show()

#***Trigram***

In [ ]:
# Vectorize the processed reviews using trigrams
vectorizer = CountVectorizer(ngram_range=(3, 3))
x = vectorizer.fit_transform(df['processed_review'])

In [ ]:
# Convert the trigrams into a DataFrame
trigram_df = pd.DataFrame(x.toarray(), columns=vectorizer.get_feature_names_out())

In [ ]:
trigram_df

In [ ]:
# Merge the trigram features with the original DataFrame
df_trigrams = pd.concat([df, trigram_df], axis=1)

In [ ]:
# Define the feature columns, including trigram features
#trigram_features = trigram_df.columns.tolist()

In [ ]:
# Calculate the sum of each trigram for AI reviews
trigram_ai_sums = df_trigrams[df_trigrams['label_encoded'] == 1][trigram_df.columns].sum().sort_values(ascending=False)

# Calculate the sum of each trigram for OR reviews
trigram_or_sums = df_trigrams[df_trigrams['label_encoded'] == 0][trigram_df.columns].sum().sort_values(ascending=False)

# Select the top 10 trigrams for AI reviews
top_n_trigrams_ai = trigram_ai_sums.head(20)

# Select the top 10 trigrams for OR reviews
top_n_trigrams_or = trigram_or_sums.head(20)

In [ ]:
# Plot the top 10 trigrams for OR reviews
plt.figure(figsize=(10, 8))
sns.barplot(x=top_n_trigrams_or.values, y=top_n_trigrams_or.index, palette="Accent")
plt.title('Top 20 Most Frequent Trigrams in OR Reviews',fontweight='bold',fontstyle='italic')
plt.xlabel('Frequency')
plt.ylabel('Trigrams')
plt.show()

In [ ]:
# Plot the top 20 trigrams for AI reviews
plt.figure(figsize=(10, 8))
sns.barplot(x=top_n_trigrams_ai.values, y=top_n_trigrams_ai.index, palette="plasma")
plt.title('Top 20 Most Frequent Trigrams in AI Reviews',fontweight='bold',fontstyle='italic')
plt.xlabel('Frequency')
plt.ylabel('Trigrams')
plt.show()

#***PCA***

In [ ]:
# Standardizing the trigram features
trigram_features_standardized = StandardScaler().fit_transform(trigram_df)

In [ ]:
# Apply PCA to retain 95% of variance
pca = PCA(n_components=0.95)
trigram_pca = pca.fit_transform(trigram_features_standardized)

In [ ]:
# Convert the PCA result to a DataFrame
trigram_pca_df = pd.DataFrame(trigram_pca, columns=[f'pca_{i+1}' for i in range(trigram_pca.shape[1])])

In [ ]:
# Merge the PCA components with the original DataFrame
df = pd.concat([df_trigrams, trigram_pca_df], axis=1)

In [ ]:
pca_features = trigram_pca_df.columns.tolist()

In [ ]:
trigram_pca_df

#***Machine Learning (ML)***

In [ ]:
# feature columns
features = ['rating', 'value', 'service', 'location', 'rooms', 'cleanliness', 'sleep_quality',
            'sent_len', 'num_chars', 'avg_word_len', 'num_unique_words', 'num_nouns', 'num_verbs',
            'num_adjs','vader_neg', 'vader_neu', 'vader_pos', 'compound','textblob_polarity','textblob_subjectivity']+ pca_features

In [ ]:
target = 'label_encoded'

In [ ]:
X = df[features]
Y = df[target]

In [ ]:
# Split the data into training (40%), testing (30%), and validation (30%) sets
X_train, X_temp, y_train, y_temp = train_test_split(X, Y, test_size=0.6, random_state=42, stratify=Y)
X_test, X_valid, y_test, y_valid = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

# Print dataset sizes
print(f"Training set size: {X_train.shape[0]}")
print(f"Testing set size: {X_test.shape[0]}")
print(f"Validation set size: {X_valid.shape[0]}")

In [ ]:
# Train a Logistic Regression model
lr_model = LogisticRegression(random_state=42, max_iter=10000)
lr_model.fit(X_train, y_train)

In [ ]:
# Predict on the test set
y_test_pred = lr_model.predict(X_test)
print("Logistic Regression:")

# Evaluate the model on the test set
print("Test Set Evaluation:\n")
print(classification_report(y_test, y_test_pred))
print(f"Accuracy: {accuracy_score(y_test, y_test_pred)}\n")
print(f"Precision: {precision_score(y_test, y_test_pred, average='weighted')}\n")
print(f"Recall: {recall_score(y_test, y_test_pred, average='weighted')}\n")
print(f"F1 Score: {f1_score(y_test, y_test_pred, average='weighted')}\n")

# Confusion Matrix for Test Set
conf_matrix_test = confusion_matrix(y_test, y_test_pred)
print("Confusion Matrix - Test Set:")
print(conf_matrix_test, "\n")

# Predict on the validation set
y_valid_pred = lr_model.predict(X_valid)

# Evaluate the model on the validation set
print("Validation Set Evaluation:\n")
print(classification_report(y_valid, y_valid_pred))
print(f"Accuracy: {accuracy_score(y_valid, y_valid_pred)}\n")
print(f"Precision: {precision_score(y_valid, y_valid_pred, average='weighted')}\n")
print(f"Recall: {recall_score(y_valid, y_valid_pred, average='weighted')}\n")
print(f"F1 Score: {f1_score(y_valid, y_valid_pred, average='weighted')}\n")

# Confusion Matrix for Validation Set
conf_matrix_valid = confusion_matrix(y_valid, y_valid_pred)
print("Confusion Matrix - Validation Set:")
print(conf_matrix_valid)

In [ ]:
# Train a Decision Tree model
dt_model = DecisionTreeClassifier(random_state=42)
dt_model.fit(X_train, y_train)

In [ ]:
# Predict on the test set
y_test_pred = dt_model.predict(X_test)
print("Decision Tree:")

# Evaluate the model on the test set
print("Test Set Evaluation:\n")
print(classification_report(y_test, y_test_pred))
print(f"Accuracy: {accuracy_score(y_test, y_test_pred)}\n")
print(f"Precision: {precision_score(y_test, y_test_pred, average='weighted')}\n")
print(f"Recall: {recall_score(y_test, y_test_pred, average='weighted')}\n")
print(f"F1 Score: {f1_score(y_test, y_test_pred, average='weighted')}\n")

# Confusion Matrix for Test Set
conf_matrix_test = confusion_matrix(y_test, y_test_pred)
print("Confusion Matrix - Test Set:")
print(conf_matrix_test, "\n")

# Predict on the validation set
y_valid_pred = dt_model.predict(X_valid)

# Evaluate the model on the validation set
print("Validation Set Evaluation:\n")
print(classification_report(y_valid, y_valid_pred))
print(f"Accuracy: {accuracy_score(y_valid, y_valid_pred)}\n")
print(f"Precision: {precision_score(y_valid, y_valid_pred, average='weighted')}\n")
print(f"Recall: {recall_score(y_valid, y_valid_pred, average='weighted')}\n")
print(f"F1 Score: {f1_score(y_valid, y_valid_pred, average='weighted')}\n")

# Confusion Matrix for Validation Set
conf_matrix_valid = confusion_matrix(y_valid, y_valid_pred)
print("Confusion Matrix - Validation Set:")
print(conf_matrix_valid)

In [ ]:
conf_matrix_valid = confusion_matrix(y_valid, y_valid_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix_valid, annot=True, fmt='d', cmap='terrain', xticklabels=['OR', 'AI'], yticklabels=['OR', 'AI'])
plt.title("Confusion Matrix",fontweight='bold',fontstyle='italic')
plt.xlabel("Predicted Label",fontweight='bold',fontstyle='italic')
plt.ylabel("True Label",fontweight='bold',fontstyle='italic')
plt.show()

In [ ]:
svc_model = SVC(random_state=42)
svc_model.fit(X_train, y_train)

In [ ]:
# Predict on the test set
y_test_pred = svc_model.predict(X_test)
print("SVM:")

# Evaluate the model on the test set
print("Test Set Evaluation:\n")
print(classification_report(y_test, y_test_pred))
print(f"Accuracy: {accuracy_score(y_test, y_test_pred)}\n")
print(f"Precision: {precision_score(y_test, y_test_pred, average='weighted')}\n")
print(f"Recall: {recall_score(y_test, y_test_pred, average='weighted')}\n")
print(f"F1 Score: {f1_score(y_test, y_test_pred, average='weighted')}\n")

# Confusion Matrix for Test Set
conf_matrix_test = confusion_matrix(y_test, y_test_pred)
print("Confusion Matrix - Test Set:")
print(conf_matrix_test, "\n")

# Predict on the validation set
y_valid_pred = svc_model.predict(X_valid)

# Evaluate the model on the validation set
print("Validation Set Evaluation:\n")
print(classification_report(y_valid, y_valid_pred))
print(f"Accuracy: {accuracy_score(y_valid, y_valid_pred)}\n")
print(f"Precision: {precision_score(y_valid, y_valid_pred, average='weighted')}\n")
print(f"Recall: {recall_score(y_valid, y_valid_pred, average='weighted')}\n")
print(f"F1 Score: {f1_score(y_valid, y_valid_pred, average='weighted')}\n")

# Confusion Matrix for Validation Set
conf_matrix_valid = confusion_matrix(y_valid, y_valid_pred)
print("Confusion Matrix - Validation Set:")
print(conf_matrix_valid)

In [ ]:
# Initialize and train the Naive Bayes model
nb_model = GaussianNB()
nb_model.fit(X_train, y_train)

In [ ]:
# Predict on the test set
y_test_pred = nb_model.predict(X_test)
print("Gaussian Naive Bayes:")


# Evaluate the model on the test set
print("Test Set Evaluation:\n")
print(classification_report(y_test, y_test_pred))
print(f"Accuracy: {accuracy_score(y_test, y_test_pred)}\n")
print(f"Precision: {precision_score(y_test, y_test_pred, average='weighted')}\n")
print(f"Recall: {recall_score(y_test, y_test_pred, average='weighted')}\n")
print(f"F1 Score: {f1_score(y_test, y_test_pred, average='weighted')}\n")

# Confusion Matrix for Test Set
conf_matrix_test = confusion_matrix(y_test, y_test_pred)
print("Confusion Matrix - Test Set:")
print(conf_matrix_test, "\n")

# Predict on the validation set
y_valid_pred = nb_model.predict(X_valid)

# Evaluate the model on the validation set
print("Validation Set Evaluation:\n")
print(classification_report(y_valid, y_valid_pred))
print(f"Accuracy: {accuracy_score(y_valid, y_valid_pred)}\n")
print(f"Precision: {precision_score(y_valid, y_valid_pred, average='weighted')}\n")
print(f"Recall: {recall_score(y_valid, y_valid_pred, average='weighted')}\n")
print(f"F1 Score: {f1_score(y_valid, y_valid_pred, average='weighted')}\n")

# Confusion Matrix for Validation Set
conf_matrix_valid = confusion_matrix(y_valid, y_valid_pred)
print("Confusion Matrix - Validation Set:")
print(conf_matrix_valid)

#***Gradio***

In [ ]:
!pip install gradio

In [ ]:
import gradio as gr

In [ ]:
import gradio as gr
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import string
import warnings
import nltk
import numpy as np

# Download required NLTK resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('vader_lexicon')

warnings.filterwarnings('ignore')

# Load the dataset
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/RedDragon_Reviews.csv')

# Preprocess the text
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    words = word_tokenize(text)
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    return ' '.join(words)

df['processed_review'] = df['review'].apply(preprocess_text)

# Sentiment analysis
analyzer = SentimentIntensityAnalyzer()
df["Sentiment_Score"] = df["processed_review"].apply(lambda x: analyzer.polarity_scores(x)["compound"])

# Label encoding
df['label_encoded'] = df['label'].apply(lambda x: 1 if x == 'AI' else 0)

# Features
features = ['Sentiment_Score']
tfidf_vectorizer = TfidfVectorizer(ngram_range=(3, 3))  # Use trigrams

X_tfidf = tfidf_vectorizer.fit_transform(df['processed_review'])
X_tfidf_df = pd.DataFrame(X_tfidf.toarray(), columns=tfidf_vectorizer.get_feature_names_out())

# Apply PCA for dimensionality reduction
pca = PCA(n_components=100)  # Adjust n_components based on your dataset
X_pca = pca.fit_transform(X_tfidf_df)

# Convert PCA components DataFrame column names to strings
X_pca_df = pd.DataFrame(X_pca).add_prefix('pca_')

# Combine PCA components with sentiment score
X = pd.concat([df[features].reset_index(drop=True), X_pca_df.reset_index(drop=True)], axis=1)
y = df['label_encoded']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Train Decision Tree model
decision_tree_model = DecisionTreeClassifier(random_state=42)
decision_tree_model.fit(X_train, y_train)

# Function to predict if a review is AI or OR
def predict_review(Review):
    processed_review = preprocess_text(Review)
    sentiment_score = analyzer.polarity_scores(processed_review)["compound"]

    review_tfidf = tfidf_vectorizer.transform([processed_review]).toarray()
    review_pca = pca.transform(review_tfidf)

    features = np.concatenate(([sentiment_score], review_pca[0]))

    prediction = decision_tree_model.predict([features])

    if prediction == 1:
        return "<span style='color: red;font-weight: bold;font-size: 20px;'>CAUTION!!!!<br>There is a possibility that the review might be written using an AI (Fake)</span>"
    else:
        return "<span style='color: green;font-weight: bold;font-size: 20px;'>The review is written by a customer....Original Review (Not Fake)</span>"

# Create Gradio interface
iface = gr.Interface(
    fn=predict_review,
    inputs="text",
    outputs="html",
    title="<b>Review Authenticity Predictor</b>",
    description="<small><i>Please Note: As the model is not completely deployed/checked for overfitting, there could be scenarios where a few of the predictions might be incorrect.</i></small>"
)

# Launch the interface
iface.launch()

#***Named Entity Recognition (NER)***

In [ ]:
import pandas as pd
import nltk
from nltk import ne_chunk, pos_tag, word_tokenize
from nltk.corpus import stopwords
from nltk.tree import Tree
import matplotlib.pyplot as plt
from collections import Counter

# Ensure the necessary NLTK packages are downloaded
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('stopwords')

# Load your dataset (ensure the correct file path is provided)
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/RedDragon_Reviews.csv')

# Get the set of English stopwords
stop_words = set(stopwords.words('english'))
stop_words.update([
    'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your',
    'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it',
    "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this',
    'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had',
    'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until',
    'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before',
    'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',
    'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most',
    'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can',
    'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren',
    "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven',
    "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't",
    'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"
])

# Function to extract named entities using NLTK
def get_named_entities(text):
    words = word_tokenize(text)
    tags = pos_tag(words)
    chunked_nes = ne_chunk(tags)
    named_entities = []
    for chunk in chunked_nes:
        if isinstance(chunk, Tree):
            entity = " ".join([token for token, pos in chunk.leaves()])
            if entity.lower() not in stop_words:  # Filter out stopwords
                named_entities.append(entity)
    return named_entities

# For "OR" Label

# Filter the reviews based on the 'OR' label
df_or = df[df['label'] == 'OR']

# Apply the NER function to the 'review' column
df_or['nltk_named_entities'] = df_or['review'].apply(get_named_entities)

# Flatten the list of named entities
named_entities_or = [entity for sublist in df_or['nltk_named_entities'] for entity in sublist]

# Count the frequency of each named entity
entity_counts_or = Counter(named_entities_or)

# Get the top 20 most common named entities
most_common_entities_or = entity_counts_or.most_common(20)

# Extract entities and their counts for plotting without using zip
entities_or = [entity for entity, count in most_common_entities_or]
counts_or = [count for entity, count in most_common_entities_or]

# Plotting the bar chart for OR label
plt.figure(figsize=(10, 8))
plt.barh(entities_or, counts_or, color='skyblue')
plt.xlabel('Frequency')
plt.title('Top 20 Most Frequent Named Entities in Original Review (OR)',fontweight='bold',fontstyle='italic')
plt.gca().invert_yaxis()
plt.show()

In [ ]:
# For "AI" Label

# Filter the reviews based on the 'AI' label
df_ai = df[df['label'] == 'AI']

# Apply the NER function to the 'review' column
df_ai['nltk_named_entities'] = df_ai['review'].apply(get_named_entities)

# Flatten the list of named entities
named_entities_ai = [entity for sublist in df_ai['nltk_named_entities'] for entity in sublist]

# Count the frequency of each named entity
entity_counts_ai = Counter(named_entities_ai)

# Get the top 20 most common named entities
most_common_entities_ai = entity_counts_ai.most_common(20)

# Separate the entities and their counts for plotting
entities_ai, counts_ai = zip(*most_common_entities_ai)

# Plotting the bar chart for AI label
plt.figure(figsize=(10, 8))
plt.barh(entities_ai, counts_ai,color='lightgreen')
plt.xlabel('Frequency')
plt.title('Top 20 Most Frequent Named Entities in AI-generated reviews',fontweight='bold',fontstyle='italic')
plt.gca().invert_yaxis()
plt.show()